In [13]:
from time import time
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Dropout, Dense, Conv2D, MaxPooling2D, Flatten
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras import regularizers

def cnn(X_train, y_train, X_test, y_test):
    time_at_start = int(time())
    conv_layers = [4, 5]
    conv_layer_sizes = [2, 4]
    dense_layers = [4, 5]
    dense_layer_sizes = [2, 4]

    EPOCHS = 50
    BATCH_SIZE = 64

    for dense_layer in dense_layers:
        for dense_layer_size in dense_layer_sizes:
                    for conv_layer in conv_layers:
                        for conv_layer_size in conv_layer_sizes:
                            name = f"{conv_layer}C{conv_layer_size}-{dense_layer}D{dense_layer_size}-{int(time())}"

                            model = Sequential()
                            model.add(Input(shape=(X_train.shape[1:])))

                            for layer in range(conv_layer - 1):
                                model.add(Conv2D(conv_layer_size, kernel_size=3, padding='valid', activation='relu'))
                                model.add(MaxPooling2D(1))
                    
                            model.add(Conv2D(conv_layer_size, kernel_size=3, padding='valid', activation='relu'))
                            model.add(MaxPooling2D(1))

                            model.add(Flatten())

                            for layer in range(dense_layer - 1):
                                model.add(Dense(dense_layer_size, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
                                model.add(Dropout(0.3))

                            model.add(Dense(5, activation='softmax'))

                            opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

                            model.compile(loss='sparse_categorical_crossentropy',
                                            optimizer=opt,
                                            metrics=['accuracy'])

                            tensorboard = TensorBoard(log_dir=f'cnniteration2logs-{time_at_start}/{name}')

                            checkpoint_filepath = f"cnniteration2models-{time_at_start}/" + name + "-{epoch:02d}-{val_accuracy:.3f}.hd5"
                            checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

                            early_stopping = EarlyStopping(monitor='val_accuracy', baseline=0.5, patience=12)

                            history = model.fit(
                                X_train, y_train,
                                batch_size=BATCH_SIZE,
                                epochs=EPOCHS,
                                validation_data=(X_test, y_test),
                                callbacks=[tensorboard, checkpoint, early_stopping]
                            )

In [14]:
import glob

jab_depth_files = [file for file in glob.glob('./numpy_data_arrays/jab/*depth.npz')]
cross_depth_files = [file for file in glob.glob('./numpy_data_arrays/cross/*depth.npz')]
left_hook_depth_files = [file for file in glob.glob('./numpy_data_arrays/left_hook/*depth.npz')]
right_hook_depth_files = [file for file in glob.glob('./numpy_data_arrays/right_hook/*depth.npz')]
random_depth_files = [file for file in glob.glob('./numpy_data_arrays/random/*depth.npz')]

In [15]:
import numpy as np
import cv2

THRESHOLD_VALUE = 5
MHI_DURATION = 2

def generate_mhi(frames):
    number_of_frames = frames.shape[0]
    height = frames.shape[1]
    width = frames.shape[2]
    SAMHI_10 = np.zeros((height, width), dtype=np.float32)

    for i in range(1, number_of_frames):
        frame = frames[i]
        frame[frame > 2200] = 0

        image_binary = frame.astype(np.uint8)

        num = 2
        image_binary_prev = np.zeros((height, width), dtype=np.uint8)
        difference = np.zeros((height, width), dtype=np.uint8)

        if i == 1:
            image_binary_prev = image_binary
        elif (i % num) == 0:
            difference = cv2.absdiff(image_binary_prev, image_binary)
            image_binary_prev = image_binary

        if i == num + 1:
            _, image_binary_diff_5 = cv2.threshold(difference, THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)
            SAMHI_10 = cv2.motempl.updateMotionHistory(image_binary_diff_5, SAMHI_10, i / number_of_frames, MHI_DURATION)

        if (i > num + 1 and i % num == 0):
            _, image_binary_diff_5 = cv2.threshold(difference, THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)
            SAMHI_10 = cv2.motempl.updateMotionHistory(image_binary_diff_5, SAMHI_10, i / number_of_frames, MHI_DURATION)


    SAMHI_10 = cv2.convertScaleAbs(SAMHI_10, alpha=255, beta=0)
    return SAMHI_10

In [16]:
import cv2
import numpy as np

X = []
y = []


for file in jab_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    mhi_data = np.expand_dims(samhi, axis=-1)
    X.append(mhi_data)
    y.append("jab")

for file in cross_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    mhi_data = np.expand_dims(samhi, axis=-1)
    X.append(mhi_data)
    y.append("cross")

for file in left_hook_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    mhi_data = np.expand_dims(samhi, axis=-1)
    X.append(mhi_data)
    y.append("left_hook")

for file in right_hook_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    mhi_data = np.expand_dims(samhi, axis=-1)
    X.append(mhi_data)
    y.append("right_hook")

for file in random_depth_files:
    file_name = file.split("\\")[-1].split('.')[0]
    depth_data = np.load(file)
    depth_frames = depth_data['arr_0']
    samhi = generate_mhi(depth_frames)
    mhi_data = np.expand_dims(samhi, axis=-1)
    X.append(mhi_data)
    y.append("random")

    # cv2.imshow("MHI", samhi)
    # cv2.imshow("Keypoint", keyP1)
    # if descriptors is not None and descriptors.shape[0] > 0:
    #     cv2.imshow("Features", descriptors)

    # bow_descriptor = bow_descriptor_extractor.compute(samhi, keypoint)

    # dataset_descriptors.append(bow_descriptor)
    # dataset_labels.append(1)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [17]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

X = np.array(X)
y = np.array(y)


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
class_labels = label_encoder.classes_

X, y = shuffle(X, y, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7)

In [18]:
cnn(X_train, y_train, X_test, y_test)

Epoch 1/50
1/1 [==============================] - ETA: 0s - loss: 1.6786 - accuracy: 0.1892
Epoch 1: val_accuracy improved from -inf to 0.15909, saving model to cnniteration2models-1686529304\4C2-4D2-1686529304-01-0.159.hd5


INFO:tensorflow:Assets written to: cnniteration2models-1686529304\4C2-4D2-1686529304-01-0.159.hd5\assets


INFO:tensorflow:Assets written to: cnniteration2models-1686529304\4C2-4D2-1686529304-01-0.159.hd5\assets


1/1 [==============================] - 6s 6s/step - loss: 1.6786 - accuracy: 0.1892 - val_loss: 1.6864 - val_accuracy: 0.1591
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 1.6697 - accuracy: 0.2973
Epoch 2: val_accuracy did not improve from 0.15909
1/1 [==============================] - 2s 2s/step - loss: 1.6697 - accuracy: 0.2973 - val_loss: 1.6876 - val_accuracy: 0.1591
Epoch 3/50
1/1 [==============================] - ETA: 0s - loss: 1.6465 - accuracy: 0.2973
Epoch 3: val_accuracy did not improve from 0.15909
1/1 [==============================] - 2s 2s/step - loss: 1.6465 - accuracy: 0.2973 - val_loss: 1.6891 - val_accuracy: 0.1591
Epoch 4/50
1/1 [==============================] - ETA: 0s - loss: 1.6659 - accuracy: 0.2973
Epoch 4: val_accuracy did not improve from 0.15909
1/1 [==============================] - 2s 2s/step - loss: 1.6659 - accuracy: 0.2973 - val_loss: 1.6848 - val_accuracy: 0.1591
Epoch 5/50
1/1 [==============================] - ETA: 0s - loss: 1

INFO:tensorflow:Assets written to: cnniteration2models-1686529304\4C4-4D2-1686529331-01-0.216.hd5\assets


...

INFO:tensorflow:Assets written to: cnniteration2models-1686529304\5C4-5D4-1686529810-01-0.159.hd5\assets


1/1 [==============================] - 8s 8s/step - loss: 1.8208 - accuracy: 0.1622 - val_loss: 1.7795 - val_accuracy: 0.1591
Epoch 2/50
1/1 [==============================] - ETA: 0s - loss: 1.7794 - accuracy: 0.2973
Epoch 2: val_accuracy improved from 0.15909 to 0.19318, saving model to cnniteration2models-1686529304\5C4-5D4-1686529810-02-0.193.hd5


INFO:tensorflow:Assets written to: cnniteration2models-1686529304\5C4-5D4-1686529810-02-0.193.hd5\assets


INFO:tensorflow:Assets written to: cnniteration2models-1686529304\5C4-5D4-1686529810-02-0.193.hd5\assets


1/1 [==============================] - 6s 6s/step - loss: 1.7794 - accuracy: 0.2973 - val_loss: 1.7787 - val_accuracy: 0.1932
Epoch 3/50
1/1 [==============================] - ETA: 0s - loss: 1.7786 - accuracy: 0.2162
Epoch 3: val_accuracy did not improve from 0.19318
1/1 [==============================] - 3s 3s/step - loss: 1.7786 - accuracy: 0.2162 - val_loss: 1.7779 - val_accuracy: 0.1932
Epoch 4/50
1/1 [==============================] - ETA: 0s - loss: 1.7777 - accuracy: 0.2162
Epoch 4: val_accuracy did not improve from 0.19318
1/1 [==============================] - 3s 3s/step - loss: 1.7777 - accuracy: 0.2162 - val_loss: 1.7771 - val_accuracy: 0.1932
Epoch 5/50
1/1 [==============================] - ETA: 0s - loss: 1.7769 - accuracy: 0.2162
Epoch 5: val_accuracy did not improve from 0.19318
1/1 [==============================] - 3s 3s/step - loss: 1.7769 - accuracy: 0.2162 - val_loss: 1.7764 - val_accuracy: 0.1932
Epoch 6/50
1/1 [==============================] - ETA: 0s - loss: 1